# Part 2: MCP Integration

# Install Packages and Import Libraries

In [1]:
# Install required packages
!pip install flask flask-cors requests termcolor pyngrok -q

print("All packages installed successfully!")
print("Installed: Flask (web server), Flask-CORS (cross-origin support), requests (HTTP client), termcolor (colored output), pyngrok (tunneling)")


All packages installed successfully!
Installed: Flask (web server), Flask-CORS (cross-origin support), requests (HTTP client), termcolor (colored output), pyngrok (tunneling)


# Database Setup

In [2]:
import sqlite3
from datetime import datetime
from pathlib import Path

DB_PATH = '/content/support.db'

class DatabaseSetup:
    """SQLite database setup for customer support system."""

    def __init__(self, db_path: str = "support.db"):
        """Initialize database connection.

        Args:
            db_path: Path to the SQLite database file
        """
        self.db_path = db_path
        self.conn = None
        self.cursor = None

    def connect(self):
        """Establish database connection."""
        self.conn = sqlite3.connect(self.db_path)
        self.conn.execute("PRAGMA foreign_keys = ON")  # Enable foreign key constraints
        self.cursor = self.conn.cursor()
        print(f"Connected to database: {self.db_path}")

    def create_tables(self):
        """Create customers and tickets tables."""

        # Create customers table
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS customers (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT NOT NULL,
                email TEXT,
                phone TEXT,
                status TEXT NOT NULL DEFAULT 'active' CHECK(status IN ('active', 'disabled')),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                updated_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)

        # Create tickets table
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS tickets (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                customer_id INTEGER NOT NULL,
                issue TEXT NOT NULL,
                status TEXT NOT NULL DEFAULT 'open' CHECK(status IN ('open', 'in_progress', 'resolved')),
                priority TEXT NOT NULL DEFAULT 'medium' CHECK(priority IN ('low', 'medium', 'high')),
                created_at DATETIME DEFAULT CURRENT_TIMESTAMP,
                FOREIGN KEY (customer_id) REFERENCES customers(id) ON DELETE CASCADE
            )
        """)

        # Create indexes for better query performance
        self.cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_customers_email ON customers(email)
        """)

        self.cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_tickets_customer_id ON tickets(customer_id)
        """)

        self.cursor.execute("""
            CREATE INDEX IF NOT EXISTS idx_tickets_status ON tickets(status)
        """)

        self.conn.commit()
        print("Tables created successfully!")

    def create_triggers(self):
        """Create triggers for automatic timestamp updates."""

        # Trigger to update updated_at on customers table
        self.cursor.execute("""
            CREATE TRIGGER IF NOT EXISTS update_customer_timestamp
            AFTER UPDATE ON customers
            FOR EACH ROW
            BEGIN
                UPDATE customers SET updated_at = CURRENT_TIMESTAMP WHERE id = NEW.id;
            END
        """)

        self.conn.commit()
        print("Triggers created successfully!")

    def insert_sample_data(self):
        """Insert sample data for testing."""

        # Sample customers (15 customers with diverse data)
        customers = [
            ("John Doe", "john.doe@example.com", "+1-555-0101", "active"),
            ("Jane Smith", "jane.smith@example.com", "+1-555-0102", "active"),
            ("Bob Johnson", "bob.johnson@example.com", "+1-555-0103", "disabled"),
            ("Alice Williams", "alice.w@techcorp.com", "+1-555-0104", "active"),
            ("Charlie Brown", "charlie.brown@email.com", "+1-555-0105", "active"),
            ("Diana Prince", "diana.prince@company.org", "+1-555-0106", "active"),
            ("Edward Norton", "e.norton@business.net", "+1-555-0107", "active"),
            ("Fiona Green", "fiona.green@startup.io", "+1-555-0108", "disabled"),
            ("George Miller", "george.m@enterprise.com", "+1-555-0109", "active"),
            ("Hannah Lee", "hannah.lee@global.com", "+1-555-0110", "active"),
            ("Isaac Newton", "isaac.n@science.edu", "+1-555-0111", "active"),
            ("Julia Roberts", "julia.r@movies.com", "+1-555-0112", "active"),
            ("Kevin Chen", "kevin.chen@tech.io", "+1-555-0113", "disabled"),
            ("Laura Martinez", "laura.m@solutions.com", "+1-555-0114", "active"),
            ("Michael Scott", "michael.scott@paper.com", "+1-555-0115", "active"),
        ]

        self.cursor.executemany("""
            INSERT INTO customers (name, email, phone, status)
            VALUES (?, ?, ?, ?)
        """, customers)

        # Sample tickets (25 tickets with various statuses and priorities)
        tickets = [
            # High priority tickets
            (1, "Cannot login to account", "open", "high"),
            (4, "Database connection timeout errors", "in_progress", "high"),
            (7, "Payment processing failing for all transactions", "open", "high"),
            (10, "Critical security vulnerability found", "in_progress", "high"),
            (14, "Website completely down", "resolved", "high"),

            # Medium priority tickets
            (1, "Password reset not working", "in_progress", "medium"),
            (2, "Profile image upload fails", "resolved", "medium"),
            (5, "Email notifications not being received", "open", "medium"),
            (6, "Dashboard loading very slowly", "in_progress", "medium"),
            (9, "Export to CSV feature broken", "open", "medium"),
            (11, "Mobile app crashes on startup", "resolved", "medium"),
            (12, "Search functionality returning wrong results", "in_progress", "medium"),
            (15, "API rate limiting too restrictive", "open", "medium"),

            # Low priority tickets
            (2, "Billing question about invoice", "resolved", "low"),
            (2, "Feature request: dark mode", "open", "low"),
            (3, "Documentation outdated for API v2", "open", "low"),
            (5, "Typo in welcome email", "resolved", "low"),
            (6, "Request for additional language support", "open", "low"),
            (9, "Font size too small on settings page", "resolved", "low"),
            (11, "Feature request: export to PDF", "open", "low"),
            (12, "Color scheme suggestion for better contrast", "open", "low"),
            (14, "Request access to beta features", "in_progress", "low"),
            (15, "Question about pricing plans", "resolved", "low"),
            (4, "Feature request: integration with Slack", "open", "low"),
            (10, "Suggestion: add keyboard shortcuts", "open", "low"),
        ]

        self.cursor.executemany("""
            INSERT INTO tickets (customer_id, issue, status, priority)
            VALUES (?, ?, ?, ?)
        """, tickets)

        self.conn.commit()
        print("Sample data inserted successfully!")
        print(f"  - {len(customers)} customers added")
        print(f"  - {len(tickets)} tickets added")

    def display_schema(self):
        """Display the database schema."""

        print("\n" + "="*60)
        print("DATABASE SCHEMA")
        print("="*60)

        # Get customers table schema
        self.cursor.execute("PRAGMA table_info(customers)")
        print("\nCUSTOMERS TABLE:")
        print("-" * 60)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<15} {row[2]:<10} {'NOT NULL' if row[3] else ''} {f'DEFAULT {row[4]}' if row[4] else ''}")

        # Get tickets table schema
        self.cursor.execute("PRAGMA table_info(tickets)")
        print("\nTICKETS TABLE:")
        print("-" * 60)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<15} {row[2]:<10} {'NOT NULL' if row[3] else ''} {f'DEFAULT {row[4]}' if row[4] else ''}")

        # Get foreign keys
        self.cursor.execute("PRAGMA foreign_key_list(tickets)")
        print("\nFOREIGN KEYS:")
        print("-" * 60)
        for row in self.cursor.fetchall():
            print(f"  tickets.{row[3]} -> {row[2]}.{row[4]}")

        print("="*60 + "\n")

    def run_sample_queries(self):
        """Execute sample queries to demonstrate database functionality."""

        print("\n" + "="*60)
        print("SAMPLE QUERIES")
        print("="*60)

        # Query 1: Get all open tickets
        print("\n1. All Open Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT t.id, c.name, t.issue, t.priority, t.created_at
            FROM tickets t
            JOIN customers c ON t.customer_id = c.id
            WHERE t.status = 'open'
            ORDER BY
                CASE t.priority
                    WHEN 'high' THEN 1
                    WHEN 'medium' THEN 2
                    WHEN 'low' THEN 3
                END, t.created_at
        """)
        for row in self.cursor.fetchall():
            print(f"  Ticket #{row[0]} | {row[1]:<20} | {row[3].upper():<6} | {row[2]}")

        # Query 2: Get all high priority tickets
        print("\n2. High Priority Tickets (Any Status):")
        print("-" * 60)
        self.cursor.execute("""
            SELECT t.id, c.name, t.issue, t.status, t.created_at
            FROM tickets t
            JOIN customers c ON t.customer_id = c.id
            WHERE t.priority = 'high'
            ORDER BY t.created_at DESC
        """)
        for row in self.cursor.fetchall():
            print(f"  Ticket #{row[0]} | {row[1]:<20} | {row[3]:<11} | {row[2]}")

        # Query 3: Customer with most tickets
        print("\n3. Customers with Most Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT c.id, c.name, c.email, COUNT(t.id) as ticket_count
            FROM customers c
            LEFT JOIN tickets t ON c.id = t.customer_id
            GROUP BY c.id, c.name, c.email
            ORDER BY ticket_count DESC
            LIMIT 5
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<25} | {row[2]:<30} | {row[3]} tickets")

        # Query 4: Tickets by status count
        print("\n4. Ticket Statistics by Status:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT status, COUNT(*) as count
            FROM tickets
            GROUP BY status
            ORDER BY count DESC
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[0]:<15} | {row[1]} tickets")

        # Query 5: Tickets by priority count
        print("\n5. Ticket Statistics by Priority:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT priority, COUNT(*) as count
            FROM tickets
            GROUP BY priority
            ORDER BY
                CASE priority
                    WHEN 'high' THEN 1
                    WHEN 'medium' THEN 2
                    WHEN 'low' THEN 3
                END
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[0]:<15} | {row[1]} tickets")

        # Query 6: Active customers with open tickets
        print("\n6. Active Customers with Open Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT DISTINCT c.id, c.name, c.email, c.phone
            FROM customers c
            JOIN tickets t ON c.id = t.customer_id
            WHERE c.status = 'active' AND t.status = 'open'
            ORDER BY c.name
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<25} | {row[2]:<30} | {row[3]}")

        # Query 7: Disabled customers
        print("\n7. Disabled Customers:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT id, name, email, phone
            FROM customers
            WHERE status = 'disabled'
            ORDER BY name
        """)
        for row in self.cursor.fetchall():
            print(f"  {row[1]:<25} | {row[2]:<30} | {row[3]}")

        # Query 8: Recent tickets (last 10)
        print("\n8. Most Recent Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT t.id, c.name, t.issue, t.status, t.priority, t.created_at
            FROM tickets t
            JOIN customers c ON t.customer_id = c.id
            ORDER BY t.created_at DESC
            LIMIT 10
        """)
        for row in self.cursor.fetchall():
            print(f"  Ticket #{row[0]} | {row[1]:<20} | {row[3]:<11} | {row[4]:<6} | {row[2][:40]}")

        # Query 9: Customers without tickets
        print("\n9. Customers Without Any Tickets:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT c.id, c.name, c.email, c.status
            FROM customers c
            LEFT JOIN tickets t ON c.id = t.customer_id
            WHERE t.id IS NULL
            ORDER BY c.name
        """)
        customers_without_tickets = self.cursor.fetchall()
        if customers_without_tickets:
            for row in customers_without_tickets:
                print(f"  {row[1]:<25} | {row[2]:<30} | {row[3]}")
        else:
            print("  (All customers have at least one ticket)")

        # Query 10: In-progress tickets with customer details
        print("\n10. In-Progress Tickets with Customer Details:")
        print("-" * 60)
        self.cursor.execute("""
            SELECT t.id, c.name, c.email, c.phone, t.issue, t.priority
            FROM tickets t
            JOIN customers c ON t.customer_id = c.id
            WHERE t.status = 'in_progress'
            ORDER BY
                CASE t.priority
                    WHEN 'high' THEN 1
                    WHEN 'medium' THEN 2
                    WHEN 'low' THEN 3
                END
        """)
        for row in self.cursor.fetchall():
            print(f"  Ticket #{row[0]} | {row[1]:<20} | {row[5].upper():<6}")
            print(f"    Email: {row[2]} | Phone: {row[3]}")
            print(f"    Issue: {row[4]}")
            print()

        print("="*60 + "\n")

    def close(self):
        """Close database connection."""
        if self.conn:
            self.conn.close()
            print("Database connection closed.")


def main():
    """Main function to setup the database."""

    # Initialize database
    db = DatabaseSetup("support.db")

    try:
        # Connect to database
        db.connect()

        # Create tables
        db.create_tables()

        # Create triggers
        db.create_triggers()

        # Display schema
        db.display_schema()

        # Ask user if they want sample data
        response = input("Would you like to insert sample data? (y/n): ").lower()
        if response == 'y':
            db.insert_sample_data()

            # Ask user if they want to run sample queries
            query_response = input("\nWould you like to run sample queries? (y/n): ").lower()
            if query_response == 'y':
                db.run_sample_queries()
            else:
                # Display sample data
                print("\nSample Customers:")
                db.cursor.execute("SELECT * FROM customers LIMIT 5")
                for row in db.cursor.fetchall():
                    print(f"  {row}")
                print(f"  ... ({db.cursor.execute('SELECT COUNT(*) FROM customers').fetchone()[0]} total)")

                print("\nSample Tickets:")
                db.cursor.execute("SELECT * FROM tickets LIMIT 5")
                for row in db.cursor.fetchall():
                    print(f"  {row}")
                print(f"  ... ({db.cursor.execute('SELECT COUNT(*) FROM tickets').fetchone()[0]} total)")

        print("\n✓ Database setup complete!")

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        db.close()


if __name__ == "__main__":
    main()


Connected to database: support.db
Tables created successfully!
Triggers created successfully!

DATABASE SCHEMA

CUSTOMERS TABLE:
------------------------------------------------------------
  id              INTEGER     
  name            TEXT       NOT NULL 
  email           TEXT        
  phone           TEXT        
  status          TEXT       NOT NULL DEFAULT 'active'
  created_at      TIMESTAMP   DEFAULT CURRENT_TIMESTAMP
  updated_at      TIMESTAMP   DEFAULT CURRENT_TIMESTAMP

TICKETS TABLE:
------------------------------------------------------------
  id              INTEGER     
  customer_id     INTEGER    NOT NULL 
  issue           TEXT       NOT NULL 
  status          TEXT       NOT NULL DEFAULT 'open'
  priority        TEXT       NOT NULL DEFAULT 'medium'
  created_at      DATETIME    DEFAULT CURRENT_TIMESTAMP

FOREIGN KEYS:
------------------------------------------------------------
  tickets.customer_id -> customers.id

Would you like to insert sample data? (y/n): y

---
# Customer Management Tools

#### Required Tools

- `get_customer(customer_id)` - uses `customers.id`
- `list_customers(status, limit)` - uses `customers.status`
- `update_customer(customer_id, data)` - uses `customers` fields
- `create_ticket(customer_id, issue, priority)` - uses `tickets` fields
- `get_customer_history(customer_id)` - uses `tickets.customer_id`

#### Database Schema

Your MCP server should maintain two main data structures:

**Customers Table:**

| Column      | Type         | Constraints                    |
|-------------|--------------|--------------------------------|
| id          | INTEGER      | PRIMARY KEY                    |
| name        | TEXT         | NOT NULL                       |
| email       | TEXT         |                                |
| phone       | TEXT         |                                |
| status      | TEXT         | 'active' or 'disabled'         |
| created_at  | TIMESTAMP    |                                |
| updated_at  | TIMESTAMP    |                                |

**Tickets Table:**

| Column      | Type         | Constraints                    |
|-------------|--------------|--------------------------------|
| id          | INTEGER      | PRIMARY KEY                    |
| customer_id | INTEGER      | FK to customers.id             |
| issue       | TEXT         | NOT NULL                       |
| status      | TEXT         | 'open', 'in_progress', 'resolved' |
| priority    | TEXT         | 'low', 'medium', 'high'        |
| created_at  | DATETIME     |                                |


In [3]:
import sqlite3
import json
from datetime import datetime
from typing import Optional, Dict, List, Any

# Database path - use 'support.db' for local, '/content/support.db' for Colab
DB_PATH = 'support.db'


def get_db_connection():
    """Create a database connection with row factory for dict-like access."""
    conn = sqlite3.connect(DB_PATH)
    conn.row_factory = sqlite3.Row  # This allows us to access columns by name
    return conn

def row_to_dict(row: sqlite3.Row) -> Dict[str, Any]:
    """Convert a SQLite row to a dictionary."""
    return {key: row[key] for key in row.keys()}

# ==================== READ OPERATIONS ====================

def get_customer(customer_id: int) -> Dict[str, Any]:
    """
    Retrieve a specific customer by ID.

    Args:
        customer_id: The unique ID of the customer

    Returns:
        Dict containing customer data or error message
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        row = cursor.fetchone()
        conn.close()

        if row:
            return {
                'success': True,
                'customer': row_to_dict(row)
            }
        else:
            return {
                'success': False,
                'error': f'Customer with ID {customer_id} not found'
            }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

def list_customers(status: Optional[str] = None, limit: int = 100) -> Dict[str, Any]:
    """
    List all customers, optionally filtered by status.

    Args:
        status: Optional filter - 'active', 'disabled', or None for all
        limit: Maximum number of customers to return (default: 100)

    Returns:
        Dict containing list of customers or error message
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        if status:
            if status not in ['active', 'disabled']:
                return {
                    'success': False,
                    'error': 'Status must be "active" or "disabled"'
                }
            cursor.execute('SELECT * FROM customers WHERE status = ? ORDER BY name LIMIT ?', (status, limit))
        else:
            cursor.execute('SELECT * FROM customers ORDER BY name LIMIT ?', (limit,))

        rows = cursor.fetchall()
        conn.close()

        customers = [row_to_dict(row) for row in rows]

        return {
            'success': True,
            'count': len(customers),
            'customers': customers
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

# ==================== UPDATE OPERATIONS ====================

def update_customer(customer_id: int, data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Update customer information with validation.

    Args:
        customer_id: The unique ID of the customer to update
        data: Dictionary with fields to update (name, email, phone, status)

    Returns:
        Dict containing updated customer data or error message
    """
    try:
        # Data validation
        valid_fields = ['name', 'email', 'phone', 'status']
        update_fields = {k: v for k, v in data.items() if k in valid_fields and v is not None}

        if not update_fields:
            return {
                'success': False,
                'error': 'No valid fields to update'
            }

        # Validate status if provided
        if 'status' in update_fields:
            if update_fields['status'] not in ['active', 'disabled']:
                return {
                    'success': False,
                    'error': "Status must be 'active' or 'disabled'"
                }

        # Validate email format if provided
        if 'email' in update_fields and update_fields['email']:
            if '@' not in update_fields['email']:
                return {
                    'success': False,
                    'error': 'Invalid email format'
                }

        conn = get_db_connection()
        cursor = conn.cursor()

        # Check if customer exists
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        if not cursor.fetchone():
            conn.close()
            return {
                'success': False,
                'error': f'Customer with ID {customer_id} not found'
            }

        # Build update query
        set_clause = ", ".join([f"{k} = ?" for k in update_fields.keys()])
        values = list(update_fields.values()) + [customer_id]

        cursor.execute(f"""
            UPDATE customers
            SET {set_clause}
            WHERE id = ?
        """, values)

        conn.commit()

        # Get updated customer
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        updated_customer = row_to_dict(cursor.fetchone())
        conn.close()

        return {
            'success': True,
            'message': f'Customer {customer_id} updated successfully',
            'customer': updated_customer
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }


def create_ticket(customer_id: int, issue: str, priority: str) -> Dict[str, Any]:
    """
    Create a new support ticket for a customer.

    Args:
        customer_id: The unique ID of the customer
        issue: Description of the issue (required, cannot be empty)
        priority: Priority level - 'low', 'medium', or 'high'

    Returns:
        Dict containing created ticket data or error message
    """
    try:
        # Validate priority
        if priority not in ['low', 'medium', 'high']:
            return {
                'success': False,
                'error': 'Priority must be "low", "medium", or "high"'
            }

        # Validate issue is not empty
        if not issue or not issue.strip():
            return {
                'success': False,
                'error': 'Issue description cannot be empty'
            }

        conn = get_db_connection()
        cursor = conn.cursor()

        # Check if customer exists
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        if not cursor.fetchone():
            conn.close()
            return {
                'success': False,
                'error': f'Customer with ID {customer_id} not found'
            }

        # Insert new ticket (status defaults to 'open' per schema)
        cursor.execute('''
            INSERT INTO tickets (customer_id, issue, priority, status)
            VALUES (?, ?, ?, 'open')
        ''', (customer_id, issue.strip(), priority))

        conn.commit()

        # Get the created ticket
        ticket_id = cursor.lastrowid
        cursor.execute('SELECT * FROM tickets WHERE id = ?', (ticket_id,))
        row = cursor.fetchone()
        conn.close()

        return {
            'success': True,
            'message': f'Ticket #{ticket_id} created successfully',
            'ticket': row_to_dict(row)
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }

def get_customer_history(customer_id: int) -> Dict[str, Any]:
    """
    Get all support tickets for a specific customer.

    Args:
        customer_id: The unique ID of the customer

    Returns:
        Dict containing list of tickets or error message
    """
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        # Check if customer exists
        cursor.execute('SELECT * FROM customers WHERE id = ?', (customer_id,))
        customer = cursor.fetchone()
        if not customer:
            conn.close()
            return {
                'success': False,
                'error': f'Customer with ID {customer_id} not found'
            }

        # Get all tickets for this customer, ordered by creation date (newest first)
        cursor.execute('''
            SELECT * FROM tickets
            WHERE customer_id = ?
            ORDER BY created_at DESC
        ''', (customer_id,))

        rows = cursor.fetchall()
        conn.close()

        tickets = [row_to_dict(row) for row in rows]

        return {
            'success': True,
            'customer_id': customer_id,
            'customer_name': customer['name'],
            'ticket_count': len(tickets),
            'tickets': tickets
        }
    except Exception as e:
        return {
            'success': False,
            'error': f'Database error: {str(e)}'
        }
# Test the functions
print("✅ Customer management functions defined successfully!")
print("\n📋 Available functions:")
print("   - get_customer(customer_id)")
print("   - list_customers(status=None)")
print("   - add_customer(name, email=None, phone=None)")
print("   - update_customer(customer_id, name=None, email=None, phone=None)")
print("   - disable_customer(customer_id)")
print("   - activate_customer(customer_id)")

# Quick test
print("\n🧪 Quick test - Fetching customer ID 1:")
result = get_customer(1)
if result['success']:
    customer = result['customer']
    print(f"   Name: {customer['name']}")
    print(f"   Email: {customer['email']}")
    print(f"   Status: {customer['status']}")


✅ Customer management functions defined successfully!

📋 Available functions:
   - get_customer(customer_id)
   - list_customers(status=None)
   - add_customer(name, email=None, phone=None)
   - update_customer(customer_id, name=None, email=None, phone=None)
   - disable_customer(customer_id)
   - activate_customer(customer_id)

🧪 Quick test - Fetching customer ID 1:
   Name: John Doe
   Email: john.doe@example.com
   Status: active


# Build MCP Server

In [4]:
from flask import Flask, request, Response, jsonify
from flask_cors import CORS
import json
import threading
import time
from typing import Dict, Any, Generator

# Create Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests

# Server state
server_thread = None
server_running = False

# MCP Protocol Implementation

# Define the tools that will be exposed via MCP
MCP_TOOLS = [
        {
        "name": "get_customer",
        "description": "Retrieve a specific customer by their ID. Returns customer details including name, email, phone, status, and timestamps.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer to retrieve"
                }
            },
            "required": ["customer_id"]
        }
    },
    {
        "name": "list_customers",
        "description": "List customers in the database, optionally filtered by status (active or disabled). Returns a list of customer records.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "status": {
                    "type": "string",
                    "enum": ["active", "disabled"],
                    "description": "Optional filter by customer status"
                },
                "limit": {
                    "type": "integer",
                    "description": "Maximum number of customers to return (default: 100)",
                    "default": 100
                }
            }
        }
    },
    {
        "name": "update_customer",
        "description": "Update an existing customer's information. Provide the customer ID and a data object with fields to update (name, email, phone, status). Validates data before updating.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer to update"
                },
                "data": {
                    "type": "object",
                    "description": "Object containing fields to update",
                    "properties": {
                        "name": {
                            "type": "string",
                            "description": "New name (optional)"
                        },
                        "email": {
                            "type": "string",
                            "description": "New email address (optional)"
                        },
                        "phone": {
                            "type": "string",
                            "description": "New phone number (optional)"
                        },
                        "status": {
                            "type": "string",
                            "enum": ["active", "disabled"],
                            "description": "New status (optional)"
                        }
                    }
                }
            },
            "required": ["customer_id", "data"]
        }
    },
    {
        "name": "create_ticket",
        "description": "Create a new support ticket for a customer. Validates that the customer exists and that priority is valid before creating the ticket.",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer creating the ticket"
                },
                "issue": {
                    "type": "string",
                    "description": "Description of the issue or problem (required, cannot be empty)"
                },
                "priority": {
                    "type": "string",
                    "enum": ["low", "medium", "high"],
                    "description": "Priority level of the ticket"
                }
            },
            "required": ["customer_id", "issue", "priority"]
        }
    },
    {
        "name": "get_customer_history",
        "description": "Get all support tickets for a specific customer. Returns the customer's complete ticket history ordered by creation date (newest first).",
        "inputSchema": {
            "type": "object",
            "properties": {
                "customer_id": {
                    "type": "integer",
                    "description": "The unique ID of the customer whose ticket history to retrieve"
                }
            },
            "required": ["customer_id"]
        }
    }
]

def create_sse_message(data: Dict[str, Any]) -> str:
    """
    Format a message for Server-Sent Events (SSE).
    SSE format: 'data: {json}\n\n'
    """
    return f"data: {json.dumps(data)}\n\n"

def handle_initialize(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle MCP initialize request.
    This is the first message in the MCP protocol handshake.
    """
    return {
        "jsonrpc": "2.0",
        "id": message.get("id"),
        "result": {
            "protocolVersion": "2024-11-05",
            "capabilities": {
                "tools": {},  # We support tools
            },
            "serverInfo": {
                "name": "customer-management-server",
                "version": "1.0.0"
            }
        }
    }

def handle_tools_list(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle tools/list request.
    Returns the list of available tools.
    """
    return {
        "jsonrpc": "2.0",
        "id": message.get("id"),
        "result": {
            "tools": MCP_TOOLS
        }
    }

def handle_tools_call(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Handle tools/call request.
    Executes the requested tool and returns the result.
    """
    params = message.get("params", {})
    tool_name = params.get("name")
    arguments = params.get("arguments", {})

    # Map tool names to functions
    tool_functions = {
        "get_customer": get_customer,
        "list_customers": list_customers,
        "update_customer": update_customer,
        "create_ticket": create_ticket,
        "get_customer_history": get_customer_history,
    }

    if tool_name not in tool_functions:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32601,
                "message": f"Tool not found: {tool_name}"
            }
        }

    try:
        # Call the tool function with the provided arguments
        result = tool_functions[tool_name](**arguments)

        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "result": {
                "content": [
                    {
                        "type": "text",
                        "text": json.dumps(result, indent=2)
                    }
                ]
            }
        }
    except Exception as e:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32603,
                "message": f"Tool execution error: {str(e)}"
            }
        }

def process_mcp_message(message: Dict[str, Any]) -> Dict[str, Any]:
    """
    Process an MCP message and route it to the appropriate handler.
    """
    method = message.get("method")

    if method == "initialize":
        return handle_initialize(message)
    elif method == "tools/list":
        return handle_tools_list(message)
    elif method == "tools/call":
        return handle_tools_call(message)
    else:
        return {
            "jsonrpc": "2.0",
            "id": message.get("id"),
            "error": {
                "code": -32601,
                "message": f"Method not found: {method}"
            }
        }

# Flask Routes

@app.route('/mcp', methods=['POST'])
def mcp_endpoint():
    """
    Main MCP endpoint for MCP communication.
    Receives MCP messages and streams responses using Server-Sent Events.
    """
    # Get the MCP message from the request BEFORE entering the generator
    # This must be done in the request context
    message = request.get_json()

    def generate():
        try:
            print(f"📥 Received MCP message: {message.get('method')}")

            # Process the message
            response = process_mcp_message(message)

            print(f"📤 Sending MCP response")

            # Send the response as SSE
            yield create_sse_message(response)

        except Exception as e:
            error_response = {
                "jsonrpc": "2.0",
                "id": None,
                "error": {
                    "code": -32700,
                    "message": f"Parse error: {str(e)}"
                }
            }
            yield create_sse_message(error_response)

    return Response(generate(), mimetype='text/event-stream')

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint to verify server is running."""
    return jsonify({
        "status": "healthy",
        "server": "customer-management-mcp-server",
        "version": "1.0.0"
    })

print("✅ MCP Server implementation complete!")
print("\n🔧 Server features:")
print("   - MCP protocol support (2024-11-05)")
print("   - Server-Sent Events (SSE) streaming")
print(f"   - {len(MCP_TOOLS)} tools exposed")
print("   - Health check endpoint")
print("   - CORS enabled for cross-origin requests")

✅ MCP Server implementation complete!

🔧 Server features:
   - MCP protocol support (2024-11-05)
   - Server-Sent Events (SSE) streaming
   - 5 tools exposed
   - Health check endpoint
   - CORS enabled for cross-origin requests


# Start MCP Server

In [5]:
import threading
import time
import requests
from termcolor import colored
from pyngrok import ngrok
from google.colab import userdata

# Server configuration
SERVER_HOST = '127.0.0.1'
SERVER_PORT = 5000
SERVER_URL = f'http://{SERVER_HOST}:{SERVER_PORT}'

def run_server():
    """Run the Flask server in a separate thread."""
    global server_running
    server_running = True
    app.run(host=SERVER_HOST, port=SERVER_PORT, debug=False, use_reloader=False)

def start_server(use_ngrok=True):
    """Start the MCP server in a background thread."""
    global server_thread, server_running

    if server_thread and server_thread.is_alive():
        print(colored("⚠️  Server is already running!", "yellow"))
        return

    print(colored("🚀 Starting MCP server...", "cyan"))

    # Start server in background thread
    server_thread = threading.Thread(target=run_server, daemon=True)
    server_thread.start()

    # Wait for server to start
    time.sleep(2)

    # Check if server is healthy
    try:
        response = requests.get(f'{SERVER_URL}/health', timeout=5)
        if response.status_code == 200:
            print(colored("✅ MCP Server is running!", "green"))
            print(colored(f"📍 Local URL: {SERVER_URL}", "cyan"))

            # Set up ngrok tunnel if requested
            if use_ngrok:
                print(colored("\n🌐 Setting up public tunnel with ngrok...", "cyan"))
                try:
                    # Get ngrok authtoken from Colab secrets
                    try:
                        authtoken = userdata.get('NGROK_AUTHTOKEN')
                        ngrok.set_auth_token(authtoken)
                        print(colored("✅ Ngrok authenticated", "green"))
                    except Exception as e:
                        print(colored("⚠️  NGROK_AUTHTOKEN not found in Colab secrets", "yellow"))
                        print(colored("   To use ngrok:", "yellow"))
                        print(colored("   1. Get free authtoken from https://ngrok.com", "yellow"))
                        print(colored("   2. In Colab: Click 🔑 (Secrets) in left sidebar", "yellow"))
                        print(colored("   3. Add secret: Name='NGROK_AUTHTOKEN', Value=<your-token>", "yellow"))
                        print(colored("   4. Enable 'Notebook access' for the secret", "yellow"))
                        print(colored("   5. Re-run this cell", "yellow"))
                        print(colored("\n   Server is still accessible locally at " + SERVER_URL, "cyan"))
                        return

                    # Create ngrok tunnel
                    public_url = ngrok.connect(SERVER_PORT)
                    print(colored(f"✅ Public URL: {public_url}", "green", attrs=["bold"]))
                    print(colored(f"📍 MCP Endpoint: {public_url}/mcp", "green", attrs=["bold"]))
                    print(colored(f"📍 Health Check: {public_url}/health", "cyan"))
                    print()
                    print(colored("🔍 MCP Inspector Instructions:", "yellow", attrs=["bold"]))
                    print(colored("1. Run in terminal: npx @modelcontextprotocol/inspector", "yellow"))
                    print(colored("2. This will open MCP Inspector in your browser", "yellow"))
                    print(colored(f"3. Enter MCP URL: {public_url}/mcp", "yellow"))
                    print(colored("4. Click 'Connect' and test the customer management tools!", "yellow"))
                except Exception as e:
                    if "NGROK_AUTHTOKEN" not in str(e):
                        print(colored(f"⚠️  Could not set up ngrok tunnel: {e}", "yellow"))
                        print(colored("   Server is still accessible locally", "yellow"))
        else:
            print(colored("❌ Server started but health check failed", "red"))
    except Exception as e:
        print(colored(f"❌ Failed to connect to server: {e}", "red"))

def stop_server():
    """Stop the MCP server."""
    global server_running
    server_running = False
    print(colored("🛑 Server stopped", "yellow"))
    print(colored("   Note: In Colab, the thread will continue until the runtime is reset", "yellow"))

def check_server_status():
    """Check if the server is running."""
    try:
        response = requests.get(f'{SERVER_URL}/health', timeout=2)
        if response.status_code == 200:
            print(colored("✅ Server is running and healthy", "green"))
            health_data = response.json()
            print(f"   Status: {health_data['status']}")
            print(f"   Server: {health_data['server']}")
            print(f"   Version: {health_data['version']}")
            return True
        else:
            print(colored("❌ Server is not responding correctly", "red"))
            return False
    except Exception as e:
        print(colored("❌ Server is not running", "red"))
        print(f"   Error: {e}")
        return False

# Start the server
start_server()


🚀 Starting MCP server...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Dec/2025 17:52:53] "GET /health HTTP/1.1" 200 -


✅ MCP Server is running!
📍 Local URL: http://127.0.0.1:5000

🌐 Setting up public tunnel with ngrok...
✅ Ngrok authenticated
✅ Public URL: NgrokTunnel: "https://tiffiny-bulbourethral-unparsimoniously.ngrok-free.dev" -> "http://localhost:5000"
📍 MCP Endpoint: NgrokTunnel: "https://tiffiny-bulbourethral-unparsimoniously.ngrok-free.dev" -> "http://localhost:5000"/mcp
📍 Health Check: NgrokTunnel: "https://tiffiny-bulbourethral-unparsimoniously.ngrok-free.dev" -> "http://localhost:5000"/health

🔍 MCP Inspector Instructions:
1. Run in terminal: npx @modelcontextprotocol/inspector
2. This will open MCP Inspector in your browser
3. Enter MCP URL: NgrokTunnel: "https://tiffiny-bulbourethral-unparsimoniously.ngrok-free.dev" -> "http://localhost:5000"/mcp
4. Click 'Connect' and test the customer management tools!
